In [28]:
# start the spark server

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('soccer_analytics').getOrCreate()

In [29]:
events_df = spark.read.csv("../Dataset/actions.csv", header=True)
events_df.show(10)

+---+-------+---------+------------------+-------+---------+------------------+------------------+------------------+------------------+-----------------+-----------+-------+---------+---------+---------+-----------+-------------+
|_c0|game_id|period_id|      time_seconds|team_id|player_id|           start_x|           start_y|             end_x|             end_y|original_event_id|bodypart_id|type_id|result_id|action_id|type_name|result_name|bodypart_name|
+---+-------+---------+------------------+-------+---------+------------------+------------------+------------------+------------------+-----------------+-----------+-------+---------+---------+---------+-----------+-------------+
|  0|2500089|        1|2.7635970000000043|   1659|     9637|              52.5|              34.0|              63.0|              30.6|        251700146|          0|      0|        1|        0|     pass|    success|         foot|
|  1|2500089|        1|4.7613529999999855|   1659|     8351|              63

In [30]:
players_df = spark.read.csv("../Dataset/players.csv", header=True)
players_df.where(players_df.wyId == 9637).show()

+--------------------+------+---------+----------+--------+-------------+----------+------+--------------------+--------------------+----+-----+---------+---------------------+
|        passportArea|weight|firstName|middleName|lastName|currentTeamId| birthDate|height|                role|           birthArea|wyId| foot|shortName|currentNationalTeamId|
+--------------------+------+---------+----------+--------+-------------+----------+------+--------------------+--------------------+----+-----+---------+---------------------+
|{'name': 'Gambia'...|    74|   Joshua|      NULL|    King|       1659.0|1992-01-15|   180|{'code2': 'FW', '...|{'name': 'Norway'...|9637|right|  J. King|               7336.0|
+--------------------+------+---------+----------+--------+-------------+----------+------+--------------------+--------------------+----+-----+---------+---------------------+



In [39]:
from pyspark.sql.functions import concat, col, lit, count

combine_df = events_df.join(players_df, events_df.player_id == players_df.wyId, "left") \
    .where(events_df.type_name == "pass")
new_player_df = combine_df.select("player_id", (concat(col("firstName"), lit(" "), col("lastName"))).alias("playerName"))
# combine_df.printSchema()
new_player_df = new_player_df.groupBy("player_id", "playerName") \
            .agg(count("*").alias("numPasses"))
ordered_player_df = new_player_df.sort(new_player_df.numPasses, ascending=False)
ordered_player_df.show()

+---------+--------------------+---------+
|player_id|          playerName|numPasses|
+---------+--------------------+---------+
|    49876|        Granit Xhaka|     3716|
|    70086|Nicol\u00e1s Hern...|     3268|
|    21315|Jorge Luiz Frello...|     3136|
|    38021|     Kevin De Bruyne|     3045|
|    25726|   Kalidou Koulibaly|     2998|
|   105339|  Fernando Luiz Rosa|     2972|
|     3476|   Ivan Rakiti\u0107|     2914|
|    14723|          Toni Kroos|     2851|
|       48|      Jan Vertonghen|     2835|
|     8277|         Kyle Walker|     2794|
|     8317|David Josu\u00e9 ...|     2719|
|    31528|   N'Golo Kant\u00e9|     2676|
|     3346|Sergio Busquets i...|     2676|
|    21171|Marek Ham\u0161\u...|     2661|
|    25553|C\u00e9sar Azpili...|     2659|
|     3269|    Jordi Alba Ramos|     2634|
|    70122|  Nemanja Mati\u0107|     2586|
|   210044|          Eric  Dier|     2581|
|     3306|Sergio Ramos Garc...|     2569|
|   131959|                NULL|     2536|
+---------+